In [0]:
import copy
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, transforms

In [0]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

In [0]:
train_transform = transforms.Compose([
  transforms.Resize(224),
  transforms.RandomHorizontalFlip(),
  transforms.ToTensor(),
  transforms.Normalize(mean = mean, std = std)                                      
]) 

In [0]:
test_transform = transforms.Compose([
  transforms.Resize(224),
  transforms.ToTensor(),
  transforms.Normalize(mean = mean, std = std)                                      
])

In [0]:
# !unzip '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset.zip' -d '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/'

In [0]:
train_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/train-cropped"
test_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/test-cropped"
valid_dir = "/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset/valid-cropped"
batch_size = 16
num_workers = 2

In [57]:
cd '/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset'

/content/drive/My Drive/Deep_Learning_Projects/Currency_Dataset/CurrencyDataset


In [0]:
train_set = datasets.ImageFolder("train-cropped", transform = train_transform)

In [59]:
train_set.classes

['10', '100', '20', '200', '2000', '5', '50', '500', 'none']

In [60]:
type(train_set)

torchvision.datasets.folder.ImageFolder

In [0]:
val_set = datasets.ImageFolder("valid-cropped", transform = test_transform)

In [62]:
val_set

Dataset ImageFolder
    Number of datapoints: 871
    Root location: valid-cropped
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
           )

In [63]:
val_set.classes

['10', '100', '20', '200', '2000', '5', '50', '500', 'none']

In [0]:
test_set = datasets.ImageFolder("test-cropped", transform = test_transform)

In [65]:
test_set

Dataset ImageFolder
    Number of datapoints: 294
    Root location: test-cropped
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
           )

In [0]:
trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          num_workers = num_workers)

In [0]:
val_loader = torch.utils.data.DataLoader(val_set,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = num_workers)

In [0]:
testloader = torch.utils.data.DataLoader(test_set,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = num_workers)

In [0]:
dataloaders = {
    'train' : trainloader,
    'val' : val_loader
} 

In [0]:
total_batch_sizes = {'train' : len(trainloader), 'val' : len(val_loader)}

In [0]:
dataset_sizes = {'train' : len(trainloader), 'val' : len(val_loader)}

In [72]:
dataset_sizes

{'train': 276, 'val': 55}

In [73]:
class_names = train_set.classes

print(np.size(class_names))
print(class_names)

9
['10', '100', '20', '200', '2000', '5', '50', '500', 'none']


#Import Model

In [0]:
from torchvision import models

In [0]:
model = models.resnet18(pretrained=True)

In [146]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Freezing model parameters of above model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [148]:
num_ftrs = model.fc.in_features
num_ftrs

512

In [0]:
model.fc  = nn.Sequential(
            nn.Linear(num_ftrs, 9),
            nn.LogSoftmax(dim = 1))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [151]:
print(device)

cuda:0


In [152]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
criterion_ft = nn.NLLLoss()

optimizer_ft = optim.Adam(model.parameters(), lr=0.01)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=2):

  model = model.to(device)

  for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    #Training model
    model.train()
    counter = 0
    for inputs, labels in trainloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = model.forward(inputs)

      loss = criterion(output, labels)

      loss.backward()

      optimizer.step()

      train_loss += loss.item()*inputs.size(0)

      #printing progress of training
      counter += 1
      if(counter % 100 == 0):
        print(counter, "/", len(trainloader))  
      
    #Evaluating Model
    model.eval()
    counter = 0
    with torch.no_grad():
      total = 0
      correct = 0
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)  

        output = model.forward(inputs)

        valloss = criterion(output, labels)

        val_loss += valloss.item()*inputs.size(0)

        _, predicted = torch.max(output.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      train_loss = train_loss/len(trainloader.dataset)
      valid_loss = val_loss/len(val_loader.dataset)
      # Print out the information
      print("Accuracy of the model on test images: {}%".format(100*correct/total))
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))      

In [155]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=2)

100 / 276
200 / 276
Accuracy of the model on test images: 75.54535017221585%
Epoch: 0 	Training Loss: 1.119075 	Validation Loss: 716.023660
100 / 276
200 / 276
Accuracy of the model on test images: 78.98966704936853%
Epoch: 1 	Training Loss: 0.769344 	Validation Loss: 608.015851


#Part2


In [0]:
from torchvision import models

In [0]:
model = models.resnet18(pretrained=True)

In [26]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Freezing model parameters of above model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [28]:
num_ftrs = model.fc.in_features
num_ftrs

512

In [0]:
model.fc  = nn.Sequential(nn.Linear(num_ftrs, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, 9),
                           nn.LogSoftmax(dim=1))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
print(device)

cuda:0


In [32]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [0]:
criterion_ft = nn.NLLLoss()

optimizer_ft = optim.Adam(model.parameters())

In [0]:
def train_model(model, criterion, optimizer, num_epochs=2):

  model = model.to(device)

  for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    #Training model
    model.train()
    counter = 0
    for inputs, labels in trainloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = model.forward(inputs)

      loss = criterion(output, labels)

      loss.backward()

      optimizer.step()

      train_loss += loss.item()*inputs.size(0)

      #printing progress of training
      counter += 1
      if(counter % 100 == 0):
        print(counter, "/", len(trainloader))  
      
    #Evaluating Model
    model.eval()
    counter = 0
    with torch.no_grad():
      total = 0
      correct = 0
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)  

        output = model.forward(inputs)

        valloss = criterion(output, labels)

        val_loss += valloss.item()*inputs.size(0)

        _, predicted = torch.max(output.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      train_loss = train_loss/len(trainloader.dataset)
      valid_loss = val_loss/len(val_loader.dataset)
      # Print out the information
      print("Accuracy of the model on test images: {}%".format(100*correct/total))
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))      

In [36]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=4)

100 / 276
200 / 276
Accuracy of the model on test images: 85.1894374282434%
Epoch: 0 	Training Loss: 0.518571 	Validation Loss: 387.305272
100 / 276
200 / 276
Accuracy of the model on test images: 87.14121699196326%
Epoch: 1 	Training Loss: 0.486679 	Validation Loss: 344.766821


#Part3

In [0]:
from torchvision import models

In [38]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [39]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Freezing model parameters of above model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [41]:
num_ftrs = model.fc.in_features
num_ftrs

2048

In [0]:
model.fc  = nn.Sequential(nn.Linear(num_ftrs, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, 9),
                           nn.LogSoftmax(dim=1))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [45]:
print(device)

cuda:0


In [46]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
criterion_ft = nn.NLLLoss()

optimizer_ft = optim.Adam(model.parameters())

In [0]:
def train_model(model, criterion, optimizer, num_epochs=2):

  model = model.to(device)

  for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    #Training model
    model.train()
    counter = 0
    for inputs, labels in trainloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = model.forward(inputs)

      loss = criterion(output, labels)

      loss.backward()

      optimizer.step()

      train_loss += loss.item()*inputs.size(0)

      #printing progress of training
      counter += 1
      if(counter % 100 == 0):
        print(counter, "/", len(trainloader))  
      
    #Evaluating Model
    model.eval()
    counter = 0
    with torch.no_grad():
      total = 0
      correct = 0
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)  

        output = model.forward(inputs)

        valloss = criterion(output, labels)

        val_loss += valloss.item()*inputs.size(0)

        _, predicted = torch.max(output.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      train_loss = train_loss/len(trainloader.dataset)
      valid_loss = val_loss/len(val_loader.dataset)
      # Print out the information
      print("Accuracy of the model on test images: {}%".format(100*correct/total))
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))      

In [50]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=2)

100 / 276
200 / 276
Accuracy of the model on test images: 82.548794489093%
Epoch: 0 	Training Loss: 0.493432 	Validation Loss: 432.947270
100 / 276
200 / 276
Accuracy of the model on test images: 84.73019517795638%
Epoch: 1 	Training Loss: 0.488824 	Validation Loss: 414.394419


#Part4

In [0]:
from torchvision import models

In [75]:
model = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth


In [76]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Freezing model parameters of above model

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [78]:
num_ftrs = model.fc.in_features
num_ftrs

2048

In [0]:
model.fc  = nn.Sequential(nn.Linear(num_ftrs, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, 9),
                           nn.LogSoftmax(dim=1))

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [81]:
print(device)

cuda:0


In [82]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
criterion_ft = nn.NLLLoss()

optimizer_ft = optim.Adam(model.parameters())

In [0]:
def train_model(model, criterion, optimizer, num_epochs=2):

  model = model.to(device)

  for epoch in range(num_epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0

    #Training model
    model.train()
    counter = 0
    for inputs, labels in trainloader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = model.forward(inputs)

      loss = criterion(output, labels)

      loss.backward()

      optimizer.step()

      train_loss += loss.item()*inputs.size(0)

      #printing progress of training
      counter += 1
      if(counter % 100 == 0):
        print(counter, "/", len(trainloader))  
      
    #Evaluating Model
    model.eval()
    counter = 0
    with torch.no_grad():
      total = 0
      correct = 0
      for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)  

        output = model.forward(inputs)

        valloss = criterion(output, labels)

        val_loss += valloss.item()*inputs.size(0)

        _, predicted = torch.max(output.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

      train_loss = train_loss/len(trainloader.dataset)
      valid_loss = val_loss/len(val_loader.dataset)
      # Print out the information
      print("Accuracy of the model on test images: {}%".format(100*correct/total))
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))      

In [86]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=8)

100 / 276
200 / 276
Accuracy of the model on test images: 83.46727898966705%
Epoch: 0 	Training Loss: 0.567899 	Validation Loss: 466.886043
100 / 276
200 / 276
Accuracy of the model on test images: 80.48220436280138%
Epoch: 1 	Training Loss: 0.474818 	Validation Loss: 619.920156
100 / 276
200 / 276
Accuracy of the model on test images: 87.14121699196326%
Epoch: 2 	Training Loss: 0.424787 	Validation Loss: 357.159723
100 / 276
200 / 276
Accuracy of the model on test images: 79.21928817451206%
Epoch: 3 	Training Loss: 0.410438 	Validation Loss: 533.364399
100 / 276
200 / 276
Accuracy of the model on test images: 89.55223880597015%
Epoch: 4 	Training Loss: 0.347410 	Validation Loss: 306.136133
100 / 276
200 / 276
Accuracy of the model on test images: 87.37083811710677%
Epoch: 5 	Training Loss: 0.396262 	Validation Loss: 348.382338
100 / 276
200 / 276
Accuracy of the model on test images: 88.17451205510908%
Epoch: 6 	Training Loss: 0.318412 	Validation Loss: 340.973971
100 / 276
200 / 276


In [87]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=4)


100 / 276
200 / 276
Accuracy of the model on test images: 86.45235361653272%
Epoch: 0 	Training Loss: 0.315940 	Validation Loss: 397.064238
100 / 276
200 / 276
Accuracy of the model on test images: 90.58553386911596%
Epoch: 1 	Training Loss: 0.289513 	Validation Loss: 253.923832
100 / 276
200 / 276
Accuracy of the model on test images: 89.20780711825488%
Epoch: 2 	Training Loss: 0.284128 	Validation Loss: 338.710166
100 / 276
200 / 276
Accuracy of the model on test images: 90.81515499425947%
Epoch: 3 	Training Loss: 0.241381 	Validation Loss: 254.636983


In [88]:
train_model(model, criterion_ft, optimizer_ft, num_epochs=6)

100 / 276
200 / 276
Accuracy of the model on test images: 87.14121699196326%
Epoch: 0 	Training Loss: 0.258634 	Validation Loss: 395.254898
100 / 276
200 / 276
Accuracy of the model on test images: 87.71526980482204%
Epoch: 1 	Training Loss: 0.249206 	Validation Loss: 353.978383
100 / 276
200 / 276
Accuracy of the model on test images: 91.61882893226176%
Epoch: 2 	Training Loss: 0.223162 	Validation Loss: 223.201265
100 / 276
200 / 276
Accuracy of the model on test images: 92.0780711825488%
Epoch: 3 	Training Loss: 0.245295 	Validation Loss: 262.951666
100 / 276
200 / 276
Accuracy of the model on test images: 91.61882893226176%
Epoch: 4 	Training Loss: 0.217120 	Validation Loss: 284.208103
100 / 276
200 / 276
Accuracy of the model on test images: 90.70034443168771%
Epoch: 5 	Training Loss: 0.193060 	Validation Loss: 237.295505
